In [2]:
import pandas as pd

# Leer el archivo CSV
base_1 = pd.read_csv('base_1.csv')

# Mostrar las primeras filas del DataFrame
print(base_1.head())

                 CURP                                              CCT  \
0  KKTQ470820MGTDAN40           Escuela Primaria Valentín Gómez Farías   
1  KKTQ470820MGTDAN40  Escuela Primaria Enrique de Olavarría y Ferrari   
2  VNQH790723MCHTPN78                             Enrique de Olavarría   
3  VNQH790723MCHTPN78                             Enrique de Olavarría   
4  VNQH790723MCHTPN78                             Enrique de Olavarría   

     LATITUD   LONGITUD  HORAS_SEMANA  
0  19.375581 -99.185979            35  
1  19.375824 -99.188210            35  
2  19.376023 -99.188159             8  
3  19.376023 -99.188159            10  
4  19.376023 -99.188159             5  


### Detección de base de plazas con jornadas mayores a 40 horas

In [3]:
# Asignar un identificador único a cada persona, en este caso la CURP
id = 'CURP' 

# Crear un diccionario para almacenar los DataFrames
dataframes_base_1 = {}

# Agrupar por id y crear un DataFrame para cada grupo
for id_, grupo in base_1.groupby(id):
    dataframes_base_1[id_] = grupo

# Mostrar los DataFrames creados
for id_, df_grupo in dataframes_base_1.items():
    print(f"DataFrame para {id_}: {df_grupo} \n")

DataFrame para EZGD960522HTLDNQ24:                   CURP                           CCT    LATITUD   LONGITUD  \
10  EZGD960522HTLDNQ24        Juana Pavón de Morelos  19.362301 -99.114351   
11  EZGD960522HTLDNQ24              Bandera Nacional  19.360487 -99.118568   
12  EZGD960522HTLDNQ24  Profr. Antonio Barbosa Heldt  19.370434 -99.130104   
13  EZGD960522HTLDNQ24            Carlos A. Carrillo  19.369264 -99.142384   

    HORAS_SEMANA  
10             8  
11             8  
12             8  
13             8   

DataFrame para GMIC250706MBSFLH83:                  CURP                          CCT    LATITUD    LONGITUD  \
5  GMIC250706MBSFLH83    Prof. Fernando Brom Rojas  19.320509  -99.123453   
6  GMIC250706MBSFLH83  Escuela Primaria José Martí  25.777518 -100.372780   
7  GMIC250706MBSFLH83                 Andres Osuna  19.480232  -99.103845   

   HORAS_SEMANA  
5            19  
6            19  
7             1   

DataFrame para KKTQ470820MGTDAN40:                  CURP   

In [4]:
# Crear una lista para almacenar los DataFrames que cumplen la condición
inconsistencias_1_list = []
base_2_list = []

# Iterar sobre los DataFrames creados
for id_, df_grupo in dataframes_base_1.items():
    # Sumar todos los valores de la columna 'HORAS_SEMANA'
    suma_horas = df_grupo['HORAS_SEMANA'].sum()
    df_grupo['TOTAL_HORAS_SEMANA'] = suma_horas
    if suma_horas > 40:
        # Agregar el DataFrame a la lista de inconsistencias
        inconsistencias_1_list.append(df_grupo)
    else:
        #print(f"La suma de las horas semanales de {id_} es menor o igual a 40: {suma_horas} \n")
        base_2_list.append(df_grupo)

# Concatenar los DataFrames de la lista en un solo DataFrame
if inconsistencias_1_list:
    inconsistencias_1 = pd.concat(inconsistencias_1_list, ignore_index=True)
else:
    inconsistencias_1 = pd.DataFrame()  # Crear un DataFrame vacío si no hay inconsistencias

# Concatenar los DataFrames de la lista en un solo DataFrame
if base_2_list:
    base_2 = pd.concat(base_2_list, ignore_index=True)
else:
    base_2 = pd.DataFrame()  # Crear un DataFrame vacío si no hay inconsistencias

# Guardar el DataFrame df_out en un archivo CSV
inconsistencias_1.to_csv('inconsistencias_1.csv', index=False)
base_2.to_csv('base_2.csv', index=False)

### Detactar si las plazas corresponden al mismo CCT o a otro

In [5]:
# Leer el archivo CSV
base_2 = pd.read_csv('base_2.csv')

# Mostrar las primeras filas del DataFrame
print(base_2.head())

                 CURP                           CCT    LATITUD   LONGITUD  \
0  EZGD960522HTLDNQ24        Juana Pavón de Morelos  19.362301 -99.114351   
1  EZGD960522HTLDNQ24              Bandera Nacional  19.360487 -99.118568   
2  EZGD960522HTLDNQ24  Profr. Antonio Barbosa Heldt  19.370434 -99.130104   
3  EZGD960522HTLDNQ24            Carlos A. Carrillo  19.369264 -99.142384   
4  GMIC250706MBSFLH83     Prof. Fernando Brom Rojas  19.320509 -99.123453   

   HORAS_SEMANA  TOTAL_HORAS_SEMANA  
0             8                  32  
1             8                  32  
2             8                  32  
3             8                  32  
4            19                  39  


In [6]:
consistencias_1_list = []
base_3_list = []

# Agrupar por id y crear un DataFrame para cada grupo
for id_, grupo in base_2.groupby(id):
    num_val_unicos = grupo['CCT'].nunique()
    if num_val_unicos > 1:
        base_3_list.append(grupo)         
    else:
        consistencias_1_list.append(grupo)

# Concatenar los DataFrames de la lista en un solo DataFrame
if consistencias_1_list:
    consistencias_1 = pd.concat(consistencias_1_list, ignore_index=True)
else:
    consistencias_1 = pd.DataFrame()  # Crear un DataFrame vacío si no hay inconsistencias

# Concatenar los DataFrames de la lista en un solo DataFrame
if base_3_list:
    base_3 = pd.concat(base_3_list, ignore_index=True)
else:
    base_3 = pd.DataFrame()  # Crear un DataFrame vacío si no hay inconsistencias

# Guardar el DataFrame df_out en un archivo CSV
consistencias_1.to_csv('consistencias_1.csv', index=False)
base_3.to_csv('base_3.csv', index=False)

### Analisis geoespacial con los dos CCT más distantes entre sí

In [7]:
import requests

# FUNCIÓN PARA HACER SOLICITUDES A LA API MAPBOX ISOCHRONES
def mapbox_isochrone(base_url, routing_profile, lon, lat, mins=None, meters=None, access_token=None):
    """
    Realiza una solicitud a la API Mapbox Isochrones y devuelve un GeoDataFrame con los vértices de la isócrona.
    """
    # Construye la URL de la solicitud
    if meters is not None:
        query = f"?contours_meters={meters}"
    elif mins is not None:
        query = f"?contours_minutes={mins}"
    else:
        raise ValueError("Debes especificar 'mins' o 'meters'.")
    
    url = f"{base_url}{routing_profile}/{lon},{lat}{query}&polygons=true&denoise=1&access_token={access_token}"

    # Realiza la solicitud a la API
    response = requests.get(url)
    response.raise_for_status()  # Lanza un error si la solicitud falla
    
    # Procesa los datos recibidos
    geom = response.json()["features"][0]["geometry"]["coordinates"][0]

    # Convierte las coordenadas a un DataFrame
    coords = pd.DataFrame(geom, columns=["lon", "lat"])
    return coords


# PRUEBA DE LA FUNCIÓN
test_coords = mapbox_isochrone(
    base_url="https://api.mapbox.com/isochrone/v1/mapbox/",
    routing_profile="walking",
    lon=-99.184823289979, lat=19.33905855077223,
    meters=500,
    access_token="pk.eyJ1IjoiY2FybG9hZDEzMTciLCJhIjoiY200aXAzaG9lMDRrcDJyb3BlMWs5bjZxbyJ9.cMOjch9URfW3X133K7MDQw"
)

test_coords

,lon,lat
0,-99.185823,19.341088
1,-99.186794,19.341029
2,-99.187358,19.339593
3,-99.188514,19.339059
4,-99.189093,19.338059
5,-99.187823,19.336529
6,-99.185823,19.335395
7,-99.184823,19.335892
8,-99.183823,19.335570
9,-99.182823,19.335835


In [8]:
# Crear un diccionario para almacenar los DataFrames
dataframes_base_3 = {}

# Agrupar por id y crear un DataFrame para cada grupo
for id_, grupo in base_3.groupby(id):
    dataframes_base_3[id_] = grupo

# Mostrar los DataFrames creados
for id_, grupo in dataframes_base_3.items():
    print(f"DataFrame para {id_}: {grupo} \n")

DataFrame para EZGD960522HTLDNQ24:                  CURP                           CCT    LATITUD   LONGITUD  \
0  EZGD960522HTLDNQ24        Juana Pavón de Morelos  19.362301 -99.114351   
1  EZGD960522HTLDNQ24              Bandera Nacional  19.360487 -99.118568   
2  EZGD960522HTLDNQ24  Profr. Antonio Barbosa Heldt  19.370434 -99.130104   
3  EZGD960522HTLDNQ24            Carlos A. Carrillo  19.369264 -99.142384   

   HORAS_SEMANA  TOTAL_HORAS_SEMANA  
0             8                  32  
1             8                  32  
2             8                  32  
3             8                  32   

DataFrame para GMIC250706MBSFLH83:                  CURP                          CCT    LATITUD    LONGITUD  \
4  GMIC250706MBSFLH83    Prof. Fernando Brom Rojas  19.320509  -99.123453   
5  GMIC250706MBSFLH83  Escuela Primaria José Martí  25.777518 -100.372780   
6  GMIC250706MBSFLH83                 Andres Osuna  19.480232  -99.103845   

   HORAS_SEMANA  TOTAL_HORAS_SEMANA  
4    

In [ ]:
from itertools import combinations
from shapely.geometry import Polygon
from shapely.geometry import Point

inconsistencias_2_list = []
consistencias_2_list = []
base_4_list = []
errores_2_list = []

for id_, grupo in dataframes_base_3.items():
    v_dist = []
    for (idx, rowx), (idy, rowy) in combinations(grupo.iterrows(), 2):
        lat_1, lon_1 = rowx['LATITUD'], rowx['LONGITUD']
        lat_2, lon_2 = rowy['LATITUD'], rowy['LONGITUD']
        dist = ((lat_1 - lat_2)**2 + (lon_1 - lon_2)**2)**0.5
        v_dist.append([idx, idy, dist]) 
    v_dist_max = max(v_dist, key=lambda x: x[2])
    cct_1 = grupo.loc[v_dist_max[0], 'CCT']
    cct_2 = grupo.loc[v_dist_max[1], 'CCT']
    lon_1 = grupo.loc[v_dist_max[0], 'LONGITUD']
    lat_1 = grupo.loc[v_dist_max[0], 'LATITUD']
    lon_2 = grupo.loc[v_dist_max[1], 'LONGITUD']
    lat_2 = grupo.loc[v_dist_max[1], 'LATITUD']
    print(f'cct1 = {cct_1}, lon1 = {lon_1}, lat1 = {lat_1}')
    print(f'cct2 = {cct_2}, lon2 = {lon_2}, lat2 = {lat_2}')

    horas_dia = grupo.loc[v_dist_max[0], 'TOTAL_HORAS_SEMANA']/5
    t_desp = (15-horas_dia)*60  # Tiempo de desplazamiento en minutos
    print(f't_desp = {t_desp}')

    t_max_permitido = 60  # Tiempo máximo permitido en mapbox
    if t_desp <= t_max_permitido:
        try:
            # Obtener isócrona
            coords = mapbox_isochrone(
                base_url="https://api.mapbox.com/isochrone/v1/mapbox/",
                routing_profile="driving",
                #routing_profile="walking",
                lon=lon_1, lat=lat_1,
                mins=t_desp,  # Tiempo de conducción
                #meters=5000,  # AQUI CAMBIAR LA DISTANCIA 5km 
                access_token="pk.eyJ1IjoiY2FybG9hZDEzMTciLCJhIjoiY200aXAzaG9lMDRrcDJyb3BlMWs5bjZxbyJ9.cMOjch9URfW3X133K7MDQw"
            )
            # Convertir a polígono y añadir a la lista
            poligono = Polygon(coords.values)
        except Exception as e:
            print(f"Error al obtener la isócrona")
            errores_2_list.append(grupo)
            continue

        punto = Point(lon_2, lat_2)
        dentro = punto.within(poligono)
        if not dentro:
            inconsistencias_2_list.append(grupo)
        else:
            num_val_unicos = grupo['CCT'].nunique()
            if num_val_unicos == 2:
                consistencias_2_list.append(grupo)
            else:
                base_4_list.append(grupo)
    else:
        print(f'Tiempo de desplazamiento mayor a {t_max_permitido} minutos')
        errores_2_list.append(grupo)
        continue    
        

# Concatenar los DataFrames de la lista en un solo DataFrame
if inconsistencias_2_list:
    inconsistencias_2 = pd.concat(inconsistencias_2_list, ignore_index=True)
else:
    inconsistencias_2 = pd.DataFrame()  # Crear un DataFrame vacío si no hay inconsistencias

if consistencias_2_list:
    consistencias_2 = pd.concat(consistencias_2_list, ignore_index=True)
else:
    consistencias_2 = pd.DataFrame()  # Crear un DataFrame vacío si no hay inconsistencias

if base_4_list:
    base_4 = pd.concat(base_4_list, ignore_index=True)
else:
    base_4 = pd.DataFrame()  # Crear un DataFrame vacío si no hay inconsistencias

if errores_2_list:
    errores_2 = pd.concat(errores_2_list, ignore_index=True)
else:
    errores_2 = pd.DataFrame()  # Crear un DataFrame vacío si no hay inconsistencias

cct1 = Juana Pavón de Morelos, lon1 = -99.11435070870378, lat1 = 19.36230082093175
cct2 = Carlos A. Carrillo, lon2 = -99.14238394473897, lat2 = 19.3692636741868
t_desp = 516.0
Error al obtener la isócrona
cct1 = Prof. Fernando Brom Rojas, lon1 = -99.12345332328093, lat1 = 19.3205088928684
cct2 = Escuela Primaria José Martí, lon2 = -100.37277963560834, lat2 = 25.777517942444582
t_desp = 432.0
Error al obtener la isócrona
cct1 = Julio Dibella Barragán, lon1 = -99.13017869803508, lat1 = 19.40204735920298
cct2 = España, lon2 = -99.13205088678797, lat2 = 19.43067391439801
t_desp = 660.0
Error al obtener la isócrona
cct1 = Venustiano Carranza, lon1 = -99.23206230802282, lat1 = 19.384511592461266
cct2 = 31 y 32 Vasco de Quiroga, lon2 = -99.23936164285152, lat2 = 19.38339226220053
t_desp = 516.0
Error al obtener la isócrona


In [13]:
# Guardar el DataFrame df_out en un archivo CSV
inconsistencias_2.to_csv('inconsistencias_2.csv', index=False)
consistencias_2.to_csv('consistencias_2.csv', index=False)
base_4.to_csv('base_4.csv', index=False)
errores_2.to_csv('errores_2.csv', index=False)

### Código que falta modificar

In [3]:
import time
from shapely.geometry import Polygon

dataframe_ciclo = df  # Asignar el DataFrame que contiene los centros de trabajo

# CICLO PARA OBTENER ISÓCRONAS DE TODOS LOS CENTROS DE TRABAJO
isochrone_list = []  # Lista para guardar las isócronas
times = []  # Para estimar el tiempo de ejecución

for idx, row in dataframe_ciclo.iterrows():
    start_time = time.time()
    print(f"Obteniendo isócrona para la escuela {idx + 1} de {len(dataframe_ciclo)}...")

    try:
        # Obtener isócrona
        coords = mapbox_isochrone(
            base_url="https://api.mapbox.com/isochrone/v1/mapbox/",
            routing_profile="driving-traffic",
            #routing_profile="walking",
            lon=row["LONGITUD"], lat=row["LATITUD"],
            #mins=5,  # Tiempo de conducción
            meters=5000,  # AQUI CAMBIAR LA DISTANCIA 5km 
            access_token="pk.eyJ1IjoiY2FybG9hZDEzMTciLCJhIjoiY200aXAzaG9lMDRrcDJyb3BlMWs5bjZxbyJ9.cMOjch9URfW3X133K7MDQw"
        )
        # Convertir a polígono y añadir a la lista
        polygon = Polygon(coords.values)
        isochrone_list.append({"CCT": row["CCT"], "geometry": polygon})
    except Exception as e:
        print(f"Error al obtener la isócrona para la escuela {idx + 1}: {e}")
        continue

    # Calcular tiempo restante
    elapsed_time = time.time() - start_time
    times.append(elapsed_time)
    avg_time = sum(times) / len(times)
    remaining_time = avg_time * (len(dataframe_ciclo) - (idx + 1)) / 60
    print(f"Tiempo estimado restante: {remaining_time:.2f} minutos")

DataFrame para EZGD960522HTLDNQ24:                   CURP                           CCT    LATITUD   LONGITUD  \
9   EZGD960522HTLDNQ24        Juana Pavón de Morelos  19.362301 -99.114351   
10  EZGD960522HTLDNQ24              Bandera Nacional  19.360487 -99.118568   
11  EZGD960522HTLDNQ24  Profr. Antonio Barbosa Heldt  19.370434 -99.130104   
12  EZGD960522HTLDNQ24            Carlos A. Carrillo  19.369264 -99.142384   

     HORAS_SEMANA  
9               8  
10              8  
11              8  
12              8   

DataFrame para GMIC250706MBSFLH83:                  CURP                          CCT    LATITUD    LONGITUD  \
5  GMIC250706MBSFLH83    Prof. Fernando Brom Rojas  19.320509  -99.123453   
6  GMIC250706MBSFLH83  Escuela Primaria José Martí  25.777518 -100.372780   

    HORAS_SEMANA  
5             19  
6             19   

DataFrame para KKTQ470820MGTDAN40:                  CURP                                              CCT  \
0  KKTQ470820MGTDAN40           Escue

In [ ]:
import geopandas as gpd

# CONVERTIR A UN GEODATAFRAME
isochrones_gdf = gpd.GeoDataFrame(isochrone_list, crs="EPSG:4326")
#isochrones_gdf = gpd.GeoDataFrame(isochrone_list, crs="4326")

# IMPRIMIR BASE DE DATOS
isochrones_gdf.head()

In [4]:
# UNIR RESULTADOS CON LA BASE ORIGINAL
escuelas_gdf = gpd.GeoDataFrame(dataframe_ciclo, geometry=None)
escuelas_gdf["geometry"] = None

for idx, row in escuelas_gdf.iterrows():
    match = isochrones_gdf[isochrones_gdf["CCT"] == row["CCT"]]
    if not match.empty:
        escuelas_gdf.at[idx, "geometry"] = match["geometry"].values[0]

# IMPRIMIR BASE DE DATOS
escuelas_gdf.head()

Obteniendo isócrona para la escuela 1 de 17...
Tiempo estimado restante: 0.03 minutos
Obteniendo isócrona para la escuela 2 de 17...
Tiempo estimado restante: 0.03 minutos
Obteniendo isócrona para la escuela 3 de 17...
Tiempo estimado restante: 0.02 minutos
Obteniendo isócrona para la escuela 4 de 17...
Tiempo estimado restante: 0.02 minutos
Obteniendo isócrona para la escuela 5 de 17...
Tiempo estimado restante: 0.01 minutos
Obteniendo isócrona para la escuela 6 de 17...
Tiempo estimado restante: 0.02 minutos
Obteniendo isócrona para la escuela 7 de 17...
Tiempo estimado restante: 0.01 minutos
Obteniendo isócrona para la escuela 8 de 17...
Tiempo estimado restante: 0.01 minutos
Obteniendo isócrona para la escuela 9 de 17...
Tiempo estimado restante: 0.01 minutos
Obteniendo isócrona para la escuela 10 de 17...
Tiempo estimado restante: 0.01 minutos
Obteniendo isócrona para la escuela 11 de 17...
Tiempo estimado restante: 0.01 minutos
Obteniendo isócrona para la escuela 12 de 17...
Tiem

In [13]:
#escuelas_gdf.explore()

In [7]:
import folium

# VISUALIZAR RESULTADOS CON FOLIUM
m = folium.Map(location=[19.33905855077223, -99.184823289979], zoom_start=12)

for _, row in escuelas_gdf.iterrows():
    if row["geometry"] is not None:
        folium.Polygon(
            locations=[(p[1], p[0]) for p in list(row["geometry"].exterior.coords)],
            color="blue",
            weight=1, fill_color="blue",
            fill_opacity=0.5,
            fill=True
        ).add_to(m)
        # Añadir un punto al mapa
        folium.Marker(
            location=[row['LATITUD'], row['LONGITUD']],  # Coordenadas del punto
            popup=f"CURP: {row['CURP']}, CCT: {row['CCT']}",  # Texto emergente al hacer clic en el marcador
            icon=folium.Icon(icon='info-sign')  # Icono del marcador
        ).add_to(m)

In [19]:
# Guardar resultados
isochrones_gdf.to_file("primarias.shp")
escuelas_gdf.to_file("escuelas_con_isocronas.shp")

/home/misael/.pyenv/versions/entorno_python_3.12.3/lib/python3.12/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


In [8]:
# Mostrar el mapa interactivo
m.save("isocronas_escuelas.html")

En esta parte se programa un codigo para extraer de forma eficiente todas los centros de trabajo de una misma persona.

In [9]:
# Asignar un identificador único a cada persona, en este caso la CURP
id = 'CURP' 

# Crear un diccionario para almacenar los DataFrames
dataframes = {}

# Agrupar por id y crear un DataFrame para cada grupo
for id_, grupo in escuelas_gdf.groupby(id):
    dataframes[id_] = grupo

# Mostrar los DataFrames creados
for id_, df_grupo in dataframes.items():
    print(f"DataFrame para {id_}: {df_grupo} \n")

DataFrame para EZGD960522HTLDNQ24:                   CURP                           CCT    LATITUD   LONGITUD  \
9   EZGD960522HTLDNQ24        Juana Pavón de Morelos  19.362301 -99.114351   
10  EZGD960522HTLDNQ24              Bandera Nacional  19.360487 -99.118568   
11  EZGD960522HTLDNQ24  Profr. Antonio Barbosa Heldt  19.370434 -99.130104   
12  EZGD960522HTLDNQ24            Carlos A. Carrillo  19.369264 -99.142384   

     HORAS_SEMANA                                           geometry  
9               8  POLYGON ((-99.11335 19.40179, -99.11371 19.393...  
10              8  POLYGON ((-99.12157 19.40197, -99.12357 19.400...  
11              8  POLYGON ((-99.1261 19.40545, -99.12647 19.4038...  
12              8  POLYGON ((-99.14438 19.39838, -99.14538 19.399...   

DataFrame para GMIC250706MBSFLH83:                  CURP                          CCT    LATITUD    LONGITUD  \
5  GMIC250706MBSFLH83    Prof. Fernando Brom Rojas  19.320509  -99.123453   
6  GMIC250706MBSFLH83  Escue

In [10]:
from shapely.geometry import Point
from itertools import combinations

# Crear un DataFrame vacío con las columnas deseadas
df_out = pd.DataFrame(columns=[id, 'CCT 1', 'LONGITUD 1', 'LATITUD 1', 'geometry 1', 'CCT 2', 'LONGITUD 2', 'LATITUD 2', 'geometry 2'])

filas = []

# Mostrar los DataFrames creados
for name, df_group in dataframes.items():
    #print('Esto es ', df_group)
    for (idx, row), (idy, rowy) in combinations(df_group.iterrows(), 2):
        punto = Point(row['LONGITUD'], row['LATITUD'])
        poligono = rowy['geometry']
        print(f"Escuela del punto: {row['CCT']}")
        print(f"Escuela del poligono: {rowy['CCT']}")
        dentro = punto.within(poligono)
        print(f"dentro = {dentro}")
        if not dentro:
            filas.append({'CURP': row[id],
                           'CCT 1': row['CCT'],
                           'LONGITUD 1': row['LONGITUD'],
                           'LATITUD 1': row['LATITUD'],
                           'geometry 1': row['geometry'],
                           'CCT 2': rowy['CCT'],
                           'LONGITUD 2': rowy['LONGITUD'],
                           'LATITUD 2': rowy['LATITUD'],
                           'geometry 2': rowy['geometry'],
                           })
            
# Convertir la lista de filas en un DataFrame
df_out = pd.DataFrame(filas)

# Imprime la base el DataFrame resultante
df_out.head()

Escuela del punto: Juana Pavón de Morelos
Escuela del poligono: Bandera Nacional
dentro = True
Escuela del punto: Juana Pavón de Morelos
Escuela del poligono: Profr. Antonio Barbosa Heldt
dentro = True
Escuela del punto: Juana Pavón de Morelos
Escuela del poligono: Carlos A. Carrillo
dentro = False
Escuela del punto: Bandera Nacional
Escuela del poligono: Profr. Antonio Barbosa Heldt
dentro = True
Escuela del punto: Bandera Nacional
Escuela del poligono: Carlos A. Carrillo
dentro = True
Escuela del punto: Profr. Antonio Barbosa Heldt
Escuela del poligono: Carlos A. Carrillo
dentro = True
Escuela del punto: Prof. Fernando Brom Rojas
Escuela del poligono: Escuela Primaria José Martí
dentro = False
Escuela del punto: Escuela Primaria Valentín Gómez Farías
Escuela del poligono: Escuela Primaria Enrique de Olavarría y Ferrari
dentro = True
Escuela del punto: Julio Dibella Barragán
Escuela del poligono: España
dentro = True
Escuela del punto: Enrique de Olavarría
Escuela del poligono: Enriqu

,CURP,CCT 1,LONGITUD 1,LATITUD 1,geometry 1,CCT 2,LONGITUD 2,LATITUD 2,geometry 2
0,EZGD960522HTLDNQ24,Juana Pavón de Morelos,-99.114351,19.362301,"POLYGON ((-99.113351 19.401788, -99.113707 19....",Carlos A. Carrillo,-99.142384,19.369264,"POLYGON ((-99.144384 19.398383, -99.145384 19...."
1,GMIC250706MBSFLH83,Prof. Fernando Brom Rojas,-99.123453,19.320509,"POLYGON ((-99.120453 19.363549, -99.122453 19....",Escuela Primaria José Martí,-100.372780,25.777518,"POLYGON ((-100.39478 25.817, -100.395545 25.81..."


In [ ]:
from shapely.geometry import Point
from itertools import combinations

# Crear un DataFrame vacío con las columnas deseadas
df_out = pd.DataFrame(columns=[id, 'CCT 1', 'LONGITUD 1', 'LATITUD 1', 'geometry 1', 'CCT 2', 'LONGITUD 2', 'LATITUD 2', 'geometry 2'])

filas = []

# Mostrar los DataFrames creados
for name, df_group in dataframes.items():
    #print('Esto es ', df_group)
    for (idx, row), (idy, rowy) in combinations(df_group.iterrows(), 2):
        punto = Point(row['LONGITUD'], row['LATITUD'])
        poligono = rowy['geometry']
        print(f"Escuela del punto: {row['CCT']}")
        print(f"Escuela del poligono: {rowy['CCT']}")
        dentro = punto.within(poligono)
        print(f"dentro = {dentro}")
        if not dentro:
            filas.append({'CURP': row[id],
                           'CCT 1': row['CCT'],
                           'LONGITUD 1': row['LONGITUD'],
                           'LATITUD 1': row['LATITUD'],
                           'geometry 1': row['geometry'],
                           'CCT 2': rowy['CCT'],
                           'LONGITUD 2': rowy['LONGITUD'],
                           'LATITUD 2': rowy['LATITUD'],
                           'geometry 2': rowy['geometry'],
                           })
            
# Convertir la lista de filas en un DataFrame
df_out = pd.DataFrame(filas)

# Imprime la base el DataFrame resultante
df_out.head()

In [26]:
# Guardar el DataFrame df_out en un archivo CSV
df_out.to_csv('irregularidades.csv', index=False)

In [27]:
# Suponiendo que tienes un DataFrame llamado df y una columna llamada 'nombre_columna'
                           'LATITUD 2': rowy['LATITUD'],
                           'geometry 2': rowy['geometry'],
                           })
            
# Convertir la lista de filas en un DataFrame
df_out = pd.DataFrame(filas)

# Imprime la base el DataFrame resultante
df_out.head()

In [ ]:
# Suponiendo que tienes un DataFrame llamado df y una columna llamada 'HORAS_SEMANA'
total_horas = df['HORAS_SEMANA'].sum()
print(f"Total de horas por semana: {total_horas}")